In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re, regex
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import make_scorer, classification_report, precision_score, recall_score, f1_score, accuracy_score, jaccard_score
from sklearn.model_selection import GridSearchCV
import os


In [11]:
train_df = pd.read_csv("../data/train.csv")
val_df   = pd.read_csv("../data/val.csv")
test_df  = pd.read_csv("../data/test.csv")

In [12]:
TARGET_EMOTIONS = ['joy','sadness','anger','fear','surprise','disgust','neutral','love']
X_train, y_train = train_df["text"], train_df[TARGET_EMOTIONS]
X_val, y_val     = val_df["text"], val_df[TARGET_EMOTIONS]
X_test, y_test   = test_df["text"], test_df[TARGET_EMOTIONS]


In [13]:
tfidf = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1, 3),
    sublinear_tf=True,
    stop_words="english"
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf   = tfidf.transform(X_val)
X_test_tfidf  = tfidf.transform(X_test)

In [ ]:
X_train_tfidf.shape, X_val_tfidf.shape, X_test_tfidf.shape

In [14]:
def evaluate_model(clf, X, y, dataset_name="Dataset", get_classification_report=False):
    y_pred = clf.predict(X)

    print(f"{dataset_name} Report:")
    if get_classification_report:
        print("\nValidation Classification Report:")
        print(classification_report(y, y_pred, target_names=TARGET_EMOTIONS, digits=3))

    micro_p = precision_score(y, y_pred, average="micro")
    micro_r = recall_score(y, y_pred, average="micro")
    micro_f1 = f1_score(y, y_pred, average="micro")

    macro_p = precision_score(y, y_pred, average="macro")
    macro_r = recall_score(y, y_pred, average="macro")
    macro_f1 = f1_score(y, y_pred, average="macro")

    print(f"Micro Precision: {micro_p:.3f}, Micro Recall: {micro_r:.3f}, Micro F1: {micro_f1:.3f}")
    print(f"Macro Precision: {macro_p:.3f}, Macro Recall: {macro_r:.3f}, Macro F1: {macro_f1:.3f}")

    subset_acc = accuracy_score(y, y_pred)
    jaccard_acc = jaccard_score(y, y_pred, average="samples")

    print(f"Subset Accuracy (Exact Match): {subset_acc:.3f}")
    print(f"Jaccard Accuracy (Sample-based): {jaccard_acc:.3f}")
    return {
        "micro_precision": micro_p,
        "micro_recall": micro_r,
        "micro_f1": micro_f1,
        "macro_precision": macro_p,
        "macro_recall": macro_r,
        "macro_f1": macro_f1,
        "subset_accuracy": subset_acc,
        "jaccard_accuracy": jaccard_acc
    }

### Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)


In [16]:
base_clf = LogisticRegression()
clf = OneVsRestClassifier(base_clf, n_jobs=-1)

In [ ]:


param_grid = {
    "estimator__solver": ['liblinear', 'saga'],
    "estimator__C": [0.5, 1.0, 2.0],
    "estimator__penalty": ['l1', 'l2'],
    "estimator__class_weight": [None, 'balanced'],
    "estimator__max_iter": [500, 1000, 1500]
}
f1_micro = make_scorer(f1_score, average='micro')

grid = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring=f1_micro,
    cv=3,
    n_jobs=-1
)

grid.fit(X_train_tfidf, y_train)
print("\nGrid Search Complete.")
print("Best Params:", grid.best_params_)
print("Best Cross-Validated Micro-F1:", grid.best_score_)
best_model = grid.best_estimator_
evaluate_model(best_model, X_val_tfidf, y_val, dataset_name="Validation Set", get_classification_report=True)

best_result = evaluate_model(best_model, X_test_tfidf, y_test, dataset_name="Test Set")
result = {'Model' : 'Logistic Regression (One-vs-Rest)'}

result.update(best_result)
result_df = pd.DataFrame([result])
result_df.to_csv('../results/best_model_test_results.csv', index=False)

results = pd.DataFrame(grid.cv_results_)
os.makedirs("../results", exist_ok=True)
results.to_csv("../results/gridsearch_LR_results.csv", index=False)

/home/kaifalam/miniconda3/envs/inpaint/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/kaifalam/miniconda3/envs/inpaint/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/kaifalam/miniconda3/envs/inpaint/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/kaifalam/miniconda3/envs/inpaint/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/kaifalam/miniconda3/envs/inpaint/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warni


Grid Search Complete.
Best Params: {'estimator__C': 2.0, 'estimator__class_weight': None, 'estimator__max_iter': 500, 'estimator__penalty': 'l1', 'estimator__solver': 'liblinear'}
Best Cross-Validated Micro-F1: 0.7027992891542202
Validation Set Report:

Validation Classification Report:
              precision    recall  f1-score   support

         joy      0.730     0.405     0.521       866
     sadness      0.698     0.381     0.493       662
       anger      0.645     0.351     0.455       866
        fear      0.753     0.407     0.529       307
    surprise      0.734     0.348     0.472       554
     disgust      0.615     0.280     0.385       600
     neutral      0.816     0.890     0.851      4752
        love      0.842     0.708     0.769       641

   micro avg      0.785     0.657     0.715      9248
   macro avg      0.729     0.471     0.559      9248
weighted avg      0.765     0.657     0.688      9248
 samples avg      0.789     0.724     0.730      9248

Micro 

AttributeError: 'dict' object has no attribute 'to_csv'

### Naive Bayes

In [27]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [23]:
clf = OneVsRestClassifier(MultinomialNB())

In [25]:
param_grid = {
    "estimator": [MultinomialNB()],  
    "estimator__alpha": [0.1, 0.5, 1.0, 2.0],        
    "estimator__fit_prior": [True, False]
}
f1_micro = make_scorer(f1_score, average='micro')
grid = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring=f1_micro,
    cv=3,
    n_jobs=-1,
    verbose=0
)
grid.fit(X_train_tfidf, y_train)
print("\nGrid Search Complete.")
print("Best Params:", grid.best_params_)
print("Best Cross-Validated Micro-F1:", grid.best_score_)

best_model = grid.best_estimator_
evaluate_model(best_model, X_val_tfidf, y_val, dataset_name="Validation Set", get_classification_report=True)
best_result = evaluate_model(best_model, X_test_tfidf, y_test, dataset_name="Test Set")
result = {'Model' : 'Multinomial Naive Bayes (One-vs-Rest)'}

result.update(best_result)
result_df = pd.read_csv('../results/best_model_test_results.csv')
result_df = pd.concat([result_df, pd.DataFrame([result])], ignore_index=True)
result_df.to_csv("../results/best_model_test_results.csv", index=False)

results = pd.DataFrame(grid.cv_results_)
os.makedirs("../results", exist_ok=True)
results.to_csv("../results/gridsearch_NB_results.csv", index=False)


Grid Search Complete.
Best Params: {'estimator': MultinomialNB(), 'estimator__alpha': 0.5, 'estimator__fit_prior': True}
Best Cross-Validated Micro-F1: 0.6030505423879403
Validation Set Report:

Validation Classification Report:
              precision    recall  f1-score   support

         joy      0.869     0.099     0.178       866
     sadness      0.939     0.047     0.089       662
       anger      0.894     0.048     0.092       866
        fear      0.750     0.020     0.038       307
    surprise      0.917     0.020     0.039       554
     disgust      1.000     0.022     0.042       600
     neutral      0.746     0.961     0.840      4752
        love      0.912     0.097     0.175       641

   micro avg      0.752     0.521     0.616      9248
   macro avg      0.878     0.164     0.187      9248
weighted avg      0.823     0.521     0.482      9248
 samples avg      0.702     0.591     0.625      9248

Micro Precision: 0.752, Micro Recall: 0.521, Micro F1: 0.616
Macr

In [29]:
clf = OneVsRestClassifier(ComplementNB())
param_grid = {
    "estimator": [ComplementNB()],  
    "estimator__alpha": [0.1, 0.5, 1.0, 2.0],        
    "estimator__fit_prior": [True, False]
}
f1_micro = make_scorer(f1_score, average='micro')
grid = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring=f1_micro,
    cv=3,
    n_jobs=-1,
    verbose=0
)
grid.fit(X_train_tfidf, y_train)
print("\nGrid Search Complete.")
print("Best Params:", grid.best_params_)
print("Best Cross-Validated Micro-F1:", grid.best_score_)

best_model = grid.best_estimator_
evaluate_model(best_model, X_val_tfidf, y_val, dataset_name="Validation Set", get_classification_report=True)
best_result = evaluate_model(best_model, X_test_tfidf, y_test, dataset_name="Test Set")
result = {'Model' : 'Complement Naive Bayes (One-vs-Rest)'}

result.update(best_result)
result_df = pd.read_csv('../results/best_model_test_results.csv')
result_df = pd.concat([result_df, pd.DataFrame([result])], ignore_index=True)
result_df.to_csv("../results/best_model_test_results.csv", index=False)

results = pd.DataFrame(grid.cv_results_)
os.makedirs("../results", exist_ok=True)
results.to_csv("../results/gridsearch_NB_results.csv", index=False)


Grid Search Complete.
Best Params: {'estimator': ComplementNB(), 'estimator__alpha': 2.0, 'estimator__fit_prior': True}
Best Cross-Validated Micro-F1: 0.5810441713870821
Validation Set Report:

Validation Classification Report:
              precision    recall  f1-score   support

         joy      0.592     0.226     0.327       866
     sadness      0.483     0.169     0.251       662
       anger      0.571     0.204     0.301       866
        fear      0.270     0.130     0.176       307
    surprise      0.388     0.137     0.203       554
     disgust      0.401     0.128     0.194       600
     neutral      0.777     0.878     0.824      4752
        love      0.653     0.329     0.438       641

   micro avg      0.712     0.547     0.619      9248
   macro avg      0.517     0.275     0.339      9248
weighted avg      0.646     0.547     0.561      9248
 samples avg      0.697     0.612     0.630      9248

Micro Precision: 0.712, Micro Recall: 0.547, Micro F1: 0.619
Macro

### Linear SVM

In [ ]:
from sklearn.svm import LinearSVC, SVC

In [ ]:
clf = OneVsRestClassifier(LinearSVC())

In [ ]:
param_grid = {
    "estimator__C": [0.1, 0.5, 1.0, 2.0, 5.0],
    "estimator__class_weight": [None, "balanced"],
    "estimator__loss": ["hinge", "squared_hinge"],
    "estimator__max_iter": [1000, 1500]
}
f1_micro = make_scorer(f1_score, average="micro")

grid = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring=f1_micro,
    cv=3,
    n_jobs=-1,
    verbose=0
)
print("Starting Grid Search for Linear SVM...")
grid.fit(X_train_tfidf, y_train)
print("\nGrid Search Complete.")
print("Best Params:", grid.best_params_)
print("Best Cross-Validated Micro-F1:", grid.best_score_)

best_model = grid.best_estimator_
evaluate_model(best_model, X_val_tfidf, y_val, dataset_name="Validation Set", get_classification_report=True)
best_result = evaluate_model(best_model, X_test_tfidf, y_test, dataset_name="Test Set")
result = {'Model' : 'Linear SVM (One-vs-Rest)'}

result.update(best_result)
result_df = pd.read_csv('../results/best_model_test_results.csv')
result_df = result_df.append(result, ignore_index=True)
result_df.to_csv("../results/best_model_test_results.csv", index=False)

results = pd.DataFrame(grid.cv_results_)
os.makedirs("../results", exist_ok=True)
results.to_csv("../results/gridsearch_NB_results.csv", index=False)

clf = OneVsRestClassifier(SVC())

In [ ]:
param_grid = {
    "estimator__C": [0.1, 0.5, 1.0, 2.0, 5.0],
    "estimator__class_weight": [None, "balanced"],
    "estimator__loss": ["hinge", "squared_hinge"],
    "estimator__max_iter": [1000, 1500]
}
f1_micro = make_scorer(f1_score, average="micro")

grid = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring=f1_micro,
    cv=3,
    n_jobs=-1,
    verbose=0
)
print("Starting Grid Search for Linear SVM...")
grid.fit(X_train_tfidf, y_train)
print("\nGrid Search Complete.")
print("Best Params:", grid.best_params_)
print("Best Cross-Validated Micro-F1:", grid.best_score_)

best_model = grid.best_estimator_
evaluate_model(best_model, X_val_tfidf, y_val, dataset_name="Validation Set", get_classification_report=True)
best_result = evaluate_model(best_model, X_test_tfidf, y_test, dataset_name="Test Set")
result = {'Model' : 'Linear SVM (One-vs-Rest)'}

result.update(best_result)
result_df = pd.read_csv('../results/best_model_test_results.csv')
result_df = result_df.append(result, ignore_index=True)
result_df.to_csv("../results/best_model_test_results.csv", index=False)

results = pd.DataFrame(grid.cv_results_)
os.makedirs("../results", exist_ok=True)
results.to_csv("../results/gridsearch_NB_results.csv", index=False)